# Crunchbase: Data Cleaning

In this notebook we will be taking an initial look at the data and doing some quick cleaning.  The meat of the cleaning and wrangling will actually be done in the data wrangling notebook.

In [1]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

## Project Summary

As this project is broken up into many sections I have decided to add a quick summary to each page so readers can get an idea of the general purpose regardless of where in the project they choose to look into first.

In this project I hope to see if it is possible to predict whether or not a company will get funded based on a range of features including round data, investor data, company data, macro trends and more.  There are an endless number of factors that come into play when considering a startups success to raise capital many of which are very difficult to quantify.  This projects looks at more quantifiable data and hopes to find features that do seem to hold significance when looking at a startups ability to successfully raise a next round.

## Load Data

In [78]:
#Information about companies.
companies = pd.read_csv('data/raw/companies.csv')

#Information about investments
investments = pd.read_csv('data/raw/investments.csv')
rounds = pd.read_csv('data/raw/rounds.csv')

#All organizations: Companies, Investors, Schools
organizations = pd.read_csv('data/raw/organizations.csv')

#All people: Investors (Seed, Angel, VCs, etc...)
people = pd.read_csv('data/raw/people.csv')

#Information specific to acquisitions
acquisitions = pd.read_csv('data/raw/acquisitions.csv')

# IPO Data
ipo = pd.read_csv('data/raw/ipo.csv').drop('Unnamed: 0',1)

# Data Cleaning

Notebook is organized as follows:
1. Companies
2. Investments and Rounds
3. Organizations
4. People
5. Acquisitions
6. IPOs



# 1. Companies

## a) Basic Cleaning

In [22]:
companies.head()

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [55]:
# - to Nan Value
companies['funding_total_usd'] = companies['funding_total_usd'].replace('-', np.nan)

# String to Numeric Values
companies['funding_total_usd'] = pd.to_numeric(companies['funding_total_usd'])
companies['funding_rounds'] = pd.to_numeric(companies['funding_rounds'])

# String to Datetimes
companies['founded_at'] = pd.to_datetime(companies['founded_at'], errors = 'coerce')
companies['first_funding_at'] = pd.to_datetime(companies['first_funding_at'], errors = 'coerce')
companies['last_funding_at'] = pd.to_datetime(companies['last_funding_at'])

In [57]:
companies.shape

(51146, 14)

In [56]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51146 entries, 1 to 66366
Data columns (total 14 columns):
permalink            51146 non-null object
name                 51145 non-null object
homepage_url         48382 non-null object
category_list        49711 non-null object
funding_total_usd    41890 non-null float64
status               51146 non-null object
country_code         47442 non-null object
state_code           46331 non-null object
region               46810 non-null object
city                 46811 non-null object
funding_rounds       51146 non-null int64
founded_at           51143 non-null datetime64[ns]
first_funding_at     51123 non-null datetime64[ns]
last_funding_at      51146 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(1), object(9)
memory usage: 5.9+ MB


## b) Creating Company Categories Dataset

In [25]:
# Break up grouped up categories into list.  EX. "Apps|Cable|Distribution|Software"
def category_list_split(row):
    if type(row.category_list) == str:
        category_list = row.category_list.split('|')
        return category_list
    else:
        return str(row.category_list)

In [26]:
# Split up Categories
categories = companies.apply(category_list_split, axis=1)\
                                .apply(pd.Series)

In [27]:
# Reformat Data
company_and_category = companies[['permalink']].merge(pd.DataFrame(categories), left_index=True, right_index=True)\
                .melt(id_vars=['permalink']).drop_duplicates().dropna()
company_and_category['variable'] = 1

In [28]:
company_and_category.head()

,permalink,variable,value
0,/organization/-fame,1,Media
1,/organization/-qounter,1,Application Platforms
2,/organization/-the-one-of-them-inc-,1,Apps
3,/organization/0-6-com,1,Curated Web
4,/organization/004-technologies,1,Software


In [59]:
# Pivot so dataset has companys and categories on x and y
company_categories = company_and_category.pivot(index='permalink', columns='value', values='variable').fillna(0)
company_categories.head(3)

value,3D,3D Printing,3D Technology,Accounting,Active Lifestyle,Ad Targeting,Adaptive Equipment,Advanced Materials,Adventure Travel,Advertising,...,Wireless,Women,Writers,Young Adults,iOS,iPad,iPhone,iPod Touch,mHealth,nan
permalink,,,,,,,,,,,,,,,,,,,,,
/organization/-fame,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/organization/-qounter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/organization/-the-one-of-them-inc-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Export Data

In [79]:
#Export Company and Category Data
companies.to_csv('data/clean/companies.csv')
company_categories.to_csv('data/interim/company_categories.csv')

## 2. Investments and Rounds

Interestingly the owner of the repo choose to separate investments and rounds into separate files so we will have to first see if there are any differences between the two besides the column information and find a way to get the most out of both sets.

### a) Compare the two datasets

##### Investments the same columns as rounds plus investor information.

In [77]:
rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 12 columns):
company_permalink          114949 non-null object
company_name               114948 non-null object
company_category_list      111539 non-null object
company_country_code       106271 non-null object
company_state_code         104003 non-null object
company_region             104782 non-null object
company_city               104785 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(11)
memory usage: 10.5+ MB


In [76]:
investments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168647 entries, 0 to 168646
Data columns (total 18 columns):
company_permalink          168647 non-null object
company_name               168646 non-null object
company_category_list      165015 non-null object
company_country_code       158126 non-null object
company_state_code         155216 non-null object
company_region             156162 non-null object
company_city               156164 non-null object
investor_permalink         168635 non-null object
investor_name              168635 non-null object
investor_country_code      137708 non-null object
investor_state_code        123908 non-null object
investor_region            125430 non-null object
investor_city              136483 non-null object
funding_round_permalink    168647 non-null object
funding_round_type         168647 non-null object
funding_round_code         81062 non-null object
funded_at                  168647 non-null object
raised_amount_usd          147028 non-nu

In [7]:
#Check # of companies of investments dataframe
investment_permalinks = investments.company_permalink
investment_permalinks.drop_duplicates().count()

44739

In [8]:
#Check # of companies of rounds dataframe
rounds_permalinks = rounds.company_permalink
rounds_permalinks.drop_duplicates().count()

66368

##### Rounds and investments have different data (Rounds has more company permalinks)?

In [9]:
imp = ['company_name','investor_name','funding_round_permalink','funding_round_type','funding_round_code','funded_at']
imp2 = ['company_name','funding_round_permalink','funding_round_type','funding_round_code','funded_at']

In [10]:
investments[imp].head()

,company_name,investor_name,funding_round_permalink,funding_round_type,funding_round_code,funded_at
0,0-6.com,DT Capital Partners,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19
1,004 Technologies,VCDE Venture Partners,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24
2,01Games Technology,Cyberport Hong Kong,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,2014-07-01
3,H2O.ai,Capital One,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,2015-11-09
4,H2O.ai,Nexus Venture Partners,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,2013-05-22


In [11]:
rounds[imp2].head()

,company_name,funding_round_permalink,funding_round_type,funding_round_code,funded_at
0,#fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05
1,:Qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,2014-10-14
2,:Qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,2014-03-01
3,"(THE) ONE of THEM,Inc.",/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30
4,0-6.com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19


In [12]:
def investment_or_rounds_comp(row):
    if row.investments > row.rounds:
        return 'investments'
    else:
        return 'rounds'

In [13]:
category_comparison = pd.concat([investments.funding_round_type.value_counts(),rounds.funding_round_type.value_counts()], axis=1, sort=0)
category_comparison.columns = ['investments','rounds']
category_comparison['compared'] = category_comparison.apply(investment_or_rounds_comp, axis=1)
category_comparison

,investments,rounds,compared
venture,104157,55494,investments
seed,43747,30524,investments
angel,6493,6094,investments
undisclosed,5608,4897,investments
private_equity,2779,2285,investments
debt_financing,2197,6895,rounds
convertible_note,1590,1817,rounds
grant,1220,2200,rounds
equity_crowdfunding,325,3257,rounds
post_ipo_equity,204,638,rounds


#####  Counts are different so should combine

In [14]:
round_code_comparison = pd.concat([investments.funding_round_code.value_counts(),rounds.funding_round_code.value_counts()], axis=1, sort=0)
round_code_comparison.columns = ['investments','rounds']
round_code_comparison['compared'] = round_code_comparison.apply(investment_or_rounds_comp, axis=1)
round_code_comparison

,investments,rounds,compared
A,31436,14747,investments
B,23395,8713,investments
C,14796,4476,investments
D,7263,2025,investments
E,2851,811,investments
F,1041,283,investments
G,235,73,investments
H,45,12,investments


##### Investments dominate when looking at investment funding.  Can assume rounds has more non-standard funding types.

## b) Merge dataframes for more complete table

From 1.1) we see that the main difference between the two datasets is investor details.  
By merging the two data sets we are able to get all of the important information from both data sets.

In [15]:
investment_rounds = investments.merge(rounds, how='outer')

In [81]:
investment_rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212810 entries, 0 to 212809
Data columns (total 18 columns):
company_permalink          212810 non-null object
company_name               212809 non-null object
company_category_list      208250 non-null object
company_country_code       199495 non-null object
company_state_code         195906 non-null object
company_region             197127 non-null object
company_city               197130 non-null object
investor_permalink         168635 non-null object
investor_name              168635 non-null object
investor_country_code      137708 non-null object
investor_state_code        123908 non-null object
investor_region            125430 non-null object
investor_city              136483 non-null object
funding_round_permalink    212810 non-null object
funding_round_type         212810 non-null object
funding_round_code         85221 non-null object
funded_at                  212810 non-null object
raised_amount_usd          185857 non-nu

In [83]:
# String to numeric values
investment_rounds['raised_amount_usd'] = pd.to_numeric(investment_rounds['raised_amount_usd'])

# String to datetime values
investment_rounds['funded_at'] = pd.to_datetime(investment_rounds['funded_at'])

In [84]:
investment_rounds.shape

(212810, 18)

In [85]:
investment_rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212810 entries, 0 to 212809
Data columns (total 18 columns):
company_permalink          212810 non-null object
company_name               212809 non-null object
company_category_list      208250 non-null object
company_country_code       199495 non-null object
company_state_code         195906 non-null object
company_region             197127 non-null object
company_city               197130 non-null object
investor_permalink         168635 non-null object
investor_name              168635 non-null object
investor_country_code      137708 non-null object
investor_state_code        123908 non-null object
investor_region            125430 non-null object
investor_city              136483 non-null object
funding_round_permalink    212810 non-null object
funding_round_type         212810 non-null object
funding_round_code         85221 non-null object
funded_at                  212810 non-null datetime64[ns]
raised_amount_usd          18585

### c) Export Investment + Rounds File

In [97]:
investment_rounds.to_csv('data/interim/investment_rounds.csv')

# 3. Organizations

No Cleaning needed

In [39]:
organizations.head(3)

,crunchbase_uuid,name,type,primary_role,crunchbase_url,homepage_domain,homepage_url,profile_image_url,facebook_url,twitter_url,linkedin_url,stock_symbol,location_city,location_region,location_country_code,short_description
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,company,https://www.crunchbase.com/organization/wetpai...,wetpaint.com,http://www.wetpaint.com/,https://crunchbase-production-res.cloudinary.c...,https://www.facebook.com/Wetpaint,https://twitter.com/wetpainttv,https://www.linkedin.com/company/wetpaint,:,New York,New York,US,Wetpaint offers an online social publishing pl...
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,company,https://www.crunchbase.com/organization/zoho?u...,zoho.com,https://www.zoho.com/,https://crunchbase-production-res.cloudinary.c...,http://www.facebook.com/zoho,http://twitter.com/zoho,http://www.linkedin.com/company/zoho-corporati...,:,Pleasanton,California,US,"Zoho offers a suite of business, collaboration..."
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,company,https://www.crunchbase.com/organization/digg?u...,digg.com,http://www.digg.com,https://crunchbase-production-res.cloudinary.c...,http://www.facebook.com/digg,http://twitter.com/digg,http://www.linkedin.com/company/digg,:,New York,New York,US,Digg Inc. operates a website that enables its ...


In [64]:
organizations.shape

(606064, 16)

In [40]:
organizations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606064 entries, 0 to 606063
Data columns (total 16 columns):
crunchbase_uuid          606064 non-null object
name                     606061 non-null object
type                     606064 non-null object
primary_role             606064 non-null object
crunchbase_url           606062 non-null object
homepage_domain          606064 non-null object
homepage_url             606064 non-null object
profile_image_url        606064 non-null object
facebook_url             363769 non-null object
twitter_url              380412 non-null object
linkedin_url             272573 non-null object
stock_symbol             606064 non-null object
location_city            429452 non-null object
location_region          429355 non-null object
location_country_code    429337 non-null object
short_description        606036 non-null object
dtypes: object(16)
memory usage: 74.0+ MB


In [98]:
organizations.to_csv('data/clean/organizations.csv')

# 4. People

In [41]:
people.head(3)

,crunchbase_uuid,type,first_name,last_name,crunchbase_url,profile_image_url,facebook_url,twitter_url,linkedin_url,location_city,location_region,location_country_code,title,organization,organization_crunchbase_url
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Person,Ben,Elowitz,https://www.crunchbase.com/person/ben-elowitz?...,https://crunchbase-production-res.cloudinary.c...,http://www.facebook.com/elowitz,http://twitter.com/elowitz,http://www.linkedin.com/in/benelowitz,Seattle,Washington,US,Managing Director at Madrona Venture Group,Madrona Venture Group,https://www.crunchbase.com/organization/madron...
1,5ceca97b-493c-1446-6249-5aaa33464763,Person,Kevin,Flaherty,https://www.crunchbase.com/person/kevin-flaher...,https://crunchbase-production-res.cloudinary.c...,NaN,https://twitter.com/tallkp,http://www.linkedin.com/in/kevinflaherty,NaN,NaN,NaN,Team Member at DRSmedia,DRSmedia,https://www.crunchbase.com/organization/drs650...
2,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Person,Raju,Vegesna,https://www.crunchbase.com/person/raju-vegesna...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,https://www.linkedin.com/pub/raju-vegesna/1/65...,San Francisco,California,US,Chief Evangelist at Zoho,Zoho,https://www.crunchbase.com/organization/zoho


In [65]:
people.shape

(605630, 15)

In [42]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605630 entries, 0 to 605629
Data columns (total 15 columns):
crunchbase_uuid                605630 non-null object
type                           605630 non-null object
first_name                     605630 non-null object
last_name                      605630 non-null object
crunchbase_url                 605629 non-null object
profile_image_url              605630 non-null object
facebook_url                   54201 non-null object
twitter_url                    117772 non-null object
linkedin_url                   336341 non-null object
location_city                  342695 non-null object
location_region                347683 non-null object
location_country_code          363749 non-null object
title                          605630 non-null object
organization                   605627 non-null object
organization_crunchbase_url    605626 non-null object
dtypes: object(15)
memory usage: 69.3+ MB


#### Pretty much clean
- missing values in social media urls (Not that important for this project)
- missing values for location (Also, Not that important for this project)

In [99]:
people.to_csv('data/clean/people.csv')

# 5. Acquisitions

In [33]:
acquisitions.head(3)

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,acquirer_permalink,acquirer_name,acquirer_category_list,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquired_at,acquired_month,price_amount,price_currency_code
0,/organization/003-ru,003.RU,Consumer Electronics|Electronics|Internet,RUS,48,Moscow,Moscow,/organization/media-saturn,Media Saturn,Enterprise Software|Media|Sales and Marketing,ESP,56,Barcelona,Barcelona,2012-07-23,2012-07,NaN,USD
1,/organization/0958572-b-c-ltd,0958572 B.C. Ltd.,NaN,NaN,NaN,NaN,NaN,/organization/atlas-intellectual-property-mana...,ATLAS Intellectual Property Management Co.,Finance|FinTech|Mobile|Telecommunications,USA,WA,Seattle,Seattle,2012-02-02,2012-02,9000000.0,USD
2,/organization/1-800-communications,1-800 Communications,NaN,USA,NY,Long Island,Hicksville,/organization/carsdirect-com,CarsDirect.com,E-Commerce,USA,CA,Los Angeles,El Segundo,2005-06-12,2005-06,NaN,USD


In [60]:
acquisitions.shape

(18968, 18)

In [62]:
# String to Numeric Values
acquisitions['price_amount'] = pd.to_numeric(acquisitions['price_amount'])

# String to Datetimes
acquisitions['acquired_at'] = pd.to_datetime(acquisitions['acquired_at'], errors = 'coerce')
acquisitions['acquired_month'] = pd.to_datetime(acquisitions['acquired_month'], errors = 'coerce')

In [63]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18968 entries, 0 to 18967
Data columns (total 18 columns):
company_permalink         18968 non-null object
company_name              18968 non-null object
company_category_list     13985 non-null object
company_country_code      14300 non-null object
company_state_code        14020 non-null object
company_region            14058 non-null object
company_city              14058 non-null object
acquirer_permalink        18968 non-null object
acquirer_name             18968 non-null object
acquirer_category_list    16580 non-null object
acquirer_country_code     17798 non-null object
acquirer_state_code       17533 non-null object
acquirer_region           17609 non-null object
acquirer_city             17609 non-null object
acquired_at               18954 non-null datetime64[ns]
acquired_month            18954 non-null datetime64[ns]
price_amount              5012 non-null float64
price_currency_code       18962 non-null object
dtypes: dat

In [90]:
acquisitions.to_csv('data/clean/acquisitions.csv')

# 6. IPOs

In [68]:
ipo.head(2)

,data
0,"[1, 1, 'c:1654', NULL, 'USD', NULL, 'USD', '19..."
1,"[2, 2, 'c:1242', NULL, 'USD', NULL, NULL, '198..."


In [43]:
# reformat data
ipo.columns = ['data']
ipo['data'] = ipo['data'].str.split(',')
ipo_data = ipo['data'].apply(pd.Series).drop(columns=[13,14,15])

In [44]:
# Columns Names
ipo_data.columns = ['id','ipo_id','object_id','valuation_amount','valuation_currency_code','raised_amount','raised_currency_code','public_at','stock_symbol','source_url','source_description','created_at','updated_at']

In [92]:
ipo_data.head(2)

,id,ipo_id,object_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,source_url,source_description,created_at,updated_at
0,1,1,'c:1654',NaN,'USD',NaN,'USD',1980-12-19,'NASDAQ:AAPL',NaN,NULL,2008-02-09 05:17:45,2012-04-12 04:02:59
1,2,2,'c:1242',NaN,'USD',NaN,NaN,1986-03-13,'NASDAQ:MSFT',NaN,NULL,2008-02-09 05:25:18,2010-12-11 12:39:46


In [93]:
# Null values to np.NaN
ipo_data['valuation_amount'] = ipo_data['valuation_amount'].replace('NULL', np.nan)
ipo_data['raised_amount'] = ipo_data['raised_amount'].replace('NULL', np.nan)
ipo_data['raised_currency_code'] = ipo_data['raised_currency_code'].replace('NULL', np.nan)
ipo_data['source_url'] = ipo_data['source_url'].replace('NULL', np.nan)

# Get rid of quotes
ipo_data['object_id'] = ipo_data['object_id'].replace('\'', '')
ipo_data['valuation_currency_code'] = ipo_data['valuation_currency_code'].replace('\'', '')
ipo_data['raised_currency_code'] = ipo_data['raised_currency_code'].replace('\'', '')
ipo_data['public_at'] = ipo_data['public_at'].replace('\'', '')
ipo_data['stock_symbol'] = ipo_data['stock_symbol'].replace('\'', '')
ipo_data['created_at'] = ipo_data['created_at'].replace('\'', '')
ipo_data['updated_at'] = ipo_data['updated_at'].replace('\'', '')

# Strings to Numeric Values
ipo_data['valuation_amount'] = pd.to_numeric(ipo_data['valuation_amount'])
ipo_data['raised_amount'] = pd.to_numeric(ipo_data['raised_amount'])

# Strings to Datetime Values
ipo_data['public_at'] = pd.to_datetime(ipo_data['public_at'], errors='coerce')
ipo_data['created_at'] = pd.to_datetime(ipo_data['created_at'], errors='coerce')
ipo_data['updated_at'] = pd.to_datetime(ipo_data['updated_at'], errors='coerce')

In [94]:
ipo_data.shape

(1259, 13)

In [95]:
ipo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 13 columns):
id                         1259 non-null object
ipo_id                     1259 non-null object
object_id                  1259 non-null object
valuation_amount           108 non-null float64
valuation_currency_code    1259 non-null object
raised_amount              139 non-null float64
raised_currency_code       699 non-null object
public_at                  659 non-null datetime64[ns]
stock_symbol               1259 non-null object
source_url                 192 non-null object
source_description         1259 non-null object
created_at                 1235 non-null datetime64[ns]
updated_at                 1255 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(8)
memory usage: 127.9+ KB


In [96]:
ipo_data.to_csv('data/clean/ipo.csv')

# 7. Macro 

In [9]:
macro_gdp_xls = pd.ExcelFile('data/raw/macro_gdp_worldbank.xls')
macro_gdp_xls.sheet_names

[u'Data', u'Metadata - Countries', u'Metadata - Indicators']

In [14]:
macro_gdp = macro_gdp_xls.parse('Data')

In [15]:
macro_gdp.columns = macro_gdp.loc[2]
macro_gdp = macro_gdp.loc[3:].reset_index(drop=True)
macro_gdp.head()

2,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.243909e+10,1.585657e+10,1.780429e+10,1.990732e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.904636e+10,1.954398e+10,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192696e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.188368e+10,1.303854e+10,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09,NaN


In [17]:
macro_gdp[macro_gdp['Country Code'] == 'USA'].m

2,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
249,United States,USA,GDP (current US$),NY.GDP.MKTP.CD,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,6.858000e+11,7.437000e+11,...,1.441874e+13,1.496437e+13,1.551793e+13,1.615526e+13,1.669152e+13,1.742761e+13,1.812071e+13,1.862448e+13,1.939060e+13,NaN
